In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [39]:
cmp_df = pd.read_csv('cmp.csv', sep=';')

In [40]:
cmp_df = cmp_df.drop_duplicates('ID')

In [41]:
cmp_df

,COMPANY_TYPE,EMPLOYEES,ID,INDUSTRY,REVENUE,TITLE
0,CUSTOMER,EMPLOYEES_1,3,IT,0.0,Коммаш
1,CUSTOMER,EMPLOYEES_2,5,GOVERNMENT,0.0,Управление делами Губернатора и Правительства ...
2,CUSTOMER,EMPLOYEES_2,13,IT,0.0,"ООО ""Интерсвязь-10"""
3,CUSTOMER,EMPLOYEES_1,43,IT,0.0,ООО «КСБ Групп»
4,3,EMPLOYEES_1,59,IT,0.0,ООО «1С-Битрикс»
5,CUSTOMER,EMPLOYEES_1,119,3,0.0,ИП Морозова Людмила Николаевна skikross.ru
6,1,EMPLOYEES_1,167,3,0.0,ООО «Дайвер»
7,CUSTOMER,EMPLOYEES_1,169,3,0.0,ООО «ЭМИР–НН»
8,CUSTOMER,EMPLOYEES_1,181,5,0.0,ИП Горшенин Артем Андреевич
9,CUSTOMER,EMPLOYEES_1,195,3,0.0,"ООО ""ОРИОН РТИ"""


In [42]:
deal_df = pd.read_csv('deal.csv', sep=';')

In [43]:
deal_df

,BEGINDATE,CLOSED,CLOSEDATE,COMPANY_ID,ID,OPPORTUNITY,PROBABILITY
0,2013-04-09T03:00:00+04:00,Y,2013-04-14T03:00:00+04:00,3,7,1078.0,100.0
1,2015-03-31T03:00:00+03:00,Y,2015-05-23T03:00:00+03:00,3,2509,4350.0,0.0
2,2018-07-16T03:00:00+03:00,N,2018-08-01T03:00:00+03:00,3,14014,2000.0,NaN
3,2013-06-09T03:00:00+04:00,Y,2013-06-16T03:00:00+04:00,5,9,30000.0,80.0
4,2015-04-07T03:00:00+03:00,Y,2015-11-19T03:00:00+03:00,5,2565,89940.0,0.0
5,2015-09-07T03:00:00+03:00,Y,2018-02-13T03:00:00+03:00,5,4888,11600.0,0.0
6,2015-12-03T03:00:00+03:00,Y,2016-01-11T03:00:00+03:00,5,6414,168200.0,0.0
7,2016-06-03T03:00:00+03:00,Y,2016-11-15T03:00:00+03:00,5,7874,0.0,0.0
8,2016-12-06T03:00:00+03:00,Y,2018-02-13T03:00:00+03:00,5,7950,29000.0,0.0
9,2017-09-04T03:00:00+03:00,Y,2017-09-11T03:00:00+03:00,5,11976,40950.0,50.0


In [44]:
inv_df = pd.read_csv('inv.csv', sep=';')

In [45]:
inv_df

,DATE_BILL,DATE_PAYED,DATE_PAY_BEFORE,PAYED,PRICE,STATUS_ID,UF_COMPANY_ID
0,2013-07-18T03:00:00+04:00,2016-10-18T07:26:24+03:00,NaN,N,38525.0,P,13
1,2013-08-21T03:00:00+04:00,2013-08-28T09:08:40+04:00,NaN,Y,3800.0,P,43
2,2013-09-16T03:00:00+04:00,NaN,NaN,N,50000.0,D,59
3,2013-07-18T03:00:00+04:00,2013-11-05T10:59:44+04:00,NaN,Y,38525.0,P,13
4,2013-07-18T03:00:00+04:00,2013-11-05T10:58:54+04:00,NaN,Y,74550.0,P,13
5,2013-10-27T03:00:00+04:00,2013-11-05T10:52:49+04:00,NaN,Y,19000.0,P,13
6,2013-10-27T03:00:00+04:00,2013-12-13T14:21:53+04:00,NaN,Y,59500.0,P,13
7,2013-11-07T03:00:00+04:00,2013-11-12T15:27:30+04:00,NaN,Y,3948.0,P,119
8,2013-11-17T03:00:00+04:00,2013-12-25T08:25:05+04:00,NaN,Y,74550.0,P,13
9,2013-11-17T03:00:00+04:00,2013-12-03T19:34:45+04:00,NaN,Y,44175.0,P,13


In [46]:
quote_df = pd.read_csv('quote.csv', sep=';')

In [47]:
quote_df

,CLOSED,CLOSEDATE,COMPANY_ID,DATE_CREATE,DEAL_ID,ID,OPPORTUNITY,STATUS_ID
0,N,NaN,13,2015-12-29T08:41:00+03:00,6480.0,76,0.0,DRAFT
1,N,NaN,13,2015-12-28T13:44:18+03:00,6498.0,74,27300.0,DRAFT
2,N,2015-12-04T03:00:00+03:00,13,2015-12-01T12:08:02+03:00,5968.0,70,18900.0,RECEIVED
3,Y,2015-10-09T03:00:00+03:00,13,2015-10-06T11:00:05+03:00,5138.0,52,18900.0,DECLAINED
4,Y,2015-10-02T03:00:00+03:00,13,2015-09-29T13:01:49+03:00,5138.0,44,15750.0,APPROVED
5,Y,2015-09-20T03:00:00+03:00,13,2015-09-13T12:31:57+03:00,3761.0,42,16275.0,APPROVED
6,Y,2015-09-16T03:00:00+03:00,13,2015-09-10T11:57:46+03:00,2195.0,34,54600.0,APPROVED
7,Y,2015-09-16T03:00:00+03:00,13,2015-09-10T11:50:03+03:00,3341.0,32,4200.0,APPROVED
8,Y,2015-09-16T03:00:00+03:00,13,2015-09-10T11:09:17+03:00,1349.0,30,66150.0,APPROVED
9,Y,2015-01-29T03:00:00+03:00,13,2015-01-23T13:28:34+03:00,1955.0,3,11550.0,APPROVED
